# Konvolucione mreže

Nakon implementacije više vrsta slojeva koji se koriste u konvolucionim mrežama, zadatak je napraviti mrežu koja će trenirati na korpusu podataka CIFAR-10.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from funkcije.classifiers.cnn import *
from funkcije.data_utils import get_CIFAR10_data
from funkcije.gradient_check import eval_numerical_gradient_array, eval_numerical_gradient
from funkcije.layers import *
from funkcije.fast_layers import *
from funkcije.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) 
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ vraća relativnu grešku """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [ ]:
# Učitavanje obrađenih CIFAR10 podataka.

data = get_CIFAR10_data()
for k, v in data.items():
    print('%s: ' % k, v.shape)

# Konvolucija: Naivna implementacija
Ključna operacija konvolucionih neuralnih mreža jeste konvolucija. U `funkcije/layers.py`, implementirajte prolaz unaprijed za konvolucioni sloj unutar funkcije `conv_forward_naive`. 

Kao što i naslov kaže, kako se radi o naivnoj implementaciji efikasnost nije bitna. Važno je da kod funkcioniše.

Testirajte svoju implementaciju koristeći sledeći kod:

In [ ]:
x_shape = (2, 3, 4, 4)
w_shape = (3, 3, 4, 4)
x = np.linspace(-0.1, 0.5, num = np.prod(x_shape)).reshape(x_shape)
w = np.linspace(-0.2, 0.3, num = np.prod(w_shape)).reshape(w_shape)
b = np.linspace(-0.1, 0.2, num = 3)

conv_param = {'stride': 2, 'pad': 1}
out, _ = conv_forward_naive(x, w, b, conv_param)
correct_out = np.array([[[[-0.08759809, -0.10987781],
                           [-0.18387192, -0.2109216 ]],
                          [[ 0.21027089,  0.21661097],
                           [ 0.22847626,  0.23004637]],
                          [[ 0.50813986,  0.54309974],
                           [ 0.64082444,  0.67101435]]],
                         [[[-0.98053589, -1.03143541],
                           [-1.19128892, -1.24695841]],
                          [[ 0.69108355,  0.66880383],
                           [ 0.59480972,  0.56776003]],
                          [[ 2.36270298,  2.36904306],
                           [ 2.38090835,  2.38247847]]]])

# Uporedite vaše rezultate sa našim. Greška bi trebalo da je oko e-8
print('Testiranje conv_forward_naive')
print('razlika: ', rel_error(out, correct_out))

# Na stranu: Obrada slike pomoću konvolucije

Kako bi provjerili implementaciju i shvatili prirodu konvolucije odnosno konvolucionog sloja, pripremili smo ulaz koji sadrži svega dvije slike sa ručno pripremljenim filterima koji vrše jednostavne operacije nad slikom, pretvaranje slike u sivoskaliranu i detekciju ivice. Sloj koji ste implementirali izvršiće ove operacije, a nakon toga ćemo vizuelizovati rezultate.

In [ ]:
from scipy.misc import imread, imresize

kitten, puppy = imread('kitten.jpg'), imread('puppy.jpg')
# vršimo obradu slika kako bi imale kvadratni oblik, slika 'puppy.jpg' već ima podobne dimenzije
d = kitten.shape[1] - kitten.shape[0]
kitten_cropped = kitten[:, d//2:-d//2, :]

img_size = 200   # sve slike koje ulaze u mrežu moraju imati konzistentne dimenzije
x = np.zeros((2, 3, img_size, img_size))
x[0, :, :, :] = imresize(puppy, (img_size, img_size)).transpose((2, 0, 1))
x[1, :, :, :] = imresize(kitten_cropped, (img_size, img_size)).transpose((2, 0, 1))

# Inicijalizujemo 2 filtera, dimenzija 3x3
w = np.zeros((2, 3, 3, 3))

# Prvi filtar pretvara sliku u sivoskaliranu.
# Postavljamo vrijednosti RGB kanala filtra. Intezitet ovih vrijednosti ne treba da vas brine, odnosi se na način
# pretvaranje slike u sivoskaliranu, a zasniva se na ljudskom vidu i načinu na koji mi uočavamo boje u prirodi.
w[0, 0, :, :] = [[0, 0, 0], [0, 0.3, 0], [0, 0, 0]]
w[0, 1, :, :] = [[0, 0, 0], [0, 0.6, 0], [0, 0, 0]]
w[0, 2, :, :] = [[0, 0, 0], [0, 0.1, 0], [0, 0, 0]]

# Drugi filtar detektuje horizontalne linije u plavom kanalu. Opet, vrijednosti nisu nasumične i dolaze iz oblasti
# digitalne obrade slike. 
w[1, 2, :, :] = [[1, 2, 1], [0, 0, 0], [-1, -2, -1]]

# Nije nam potreban bias za sivoskaliranu sliku, ali jeste za detekciju ivice
# pa dodajemo 128 jer ne želimo da dobijemo negativne vrijednosti što se može dogoditi zbog prirode filtra.
b = np.array([0, 128])

# Računamo rezultat konvolucije za svaki ulaz x upotrebom filtara w,
# i dodajemo bias b, a rezultat smještamo u out. 
out, _ = conv_forward_naive(x, w, b, {'stride': 1, 'pad': 1})

def imshow_noax(img, normalize=True):
    """ Tiny helper to show images as uint8 and remove axis labels """
    if normalize:
        img_max, img_min = np.max(img), np.min(img)
        img = 255.0 * (img - img_min) / (img_max - img_min)
    plt.imshow(img.astype('uint8'))
    plt.gca().axis('off')

# Prikazujemo originalne slike i one dobijene konvolucijom.
plt.subplot(2, 3, 1)
imshow_noax(puppy, normalize = False)
plt.title('Original image')
plt.subplot(2, 3, 2)
imshow_noax(out[0, 0])
plt.title('Grayscale')
plt.subplot(2, 3, 3)
imshow_noax(out[0, 1])
plt.title('Edges')
plt.subplot(2, 3, 4)
imshow_noax(kitten_cropped, normalize=  False)
plt.subplot(2, 3, 5)
imshow_noax(out[1, 0])
plt.subplot(2, 3, 6)
imshow_noax(out[1, 1])
plt.show()

# Konvolucija: Naivni prolaz unazad
Implementirajte prolaz unazad za operaciju konvolucije unutar funkcije `conv_backward_naive` koja se nalazi u `funkcije/layers.py`. Ponovo ne treba voditi računa o efikasnosti.

Uporedite dobijene rezultate sa numeričkim izvodom.

In [ ]:
np.random.seed(231)
x = np.random.randn(4, 3, 5, 5)
w = np.random.randn(2, 3, 3, 3)
b = np.random.randn(2,)
dout = np.random.randn(4, 2, 5, 5)
conv_param = {'stride': 1, 'pad': 1}

dx_num = eval_numerical_gradient_array(lambda x: conv_forward_naive(x, w, b, conv_param)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: conv_forward_naive(x, w, b, conv_param)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: conv_forward_naive(x, w, b, conv_param)[0], b, dout)

out, cache = conv_forward_naive(x, w, b, conv_param)
dx, dw, db = conv_backward_naive(dout, cache)

# Greška bi trebalo da je e-8 ili manja.
print('Testiranje conv_backward_naive funkcije')
print('dx greška: ', rel_error(dx, dx_num))
print('dw greška: ', rel_error(dw, dw_num))
print('db greška: ', rel_error(db, db_num))

# Max-Pooling: Naivna implementacija
Implementirajte prolaz unaprijed za max-pooling operaiciju unutar funkcije `max_pool_forward_naive` koja se nalazi u `funkcije/layers.py`. Ponovo ne treba voditi računa o efikasnosti.

Testirajte svoju implementaciju koristeći sledeći kod:

In [ ]:
x_shape = (2, 3, 4, 4)
x = np.linspace(-0.3, 0.4, num = np.prod(x_shape)).reshape(x_shape)
pool_param = {'pool_width': 2, 'pool_height': 2, 'stride': 2}

out, _ = max_pool_forward_naive(x, pool_param)

correct_out = np.array([[[[-0.26315789, -0.24842105],
                          [-0.20421053, -0.18947368]],
                         [[-0.14526316, -0.13052632],
                          [-0.08631579, -0.07157895]],
                         [[-0.02736842, -0.01263158],
                          [ 0.03157895,  0.04631579]]],
                        [[[ 0.09052632,  0.10526316],
                          [ 0.14947368,  0.16421053]],
                         [[ 0.20842105,  0.22315789],
                          [ 0.26736842,  0.28210526]],
                         [[ 0.32631579,  0.34105263],
                          [ 0.38526316,  0.4       ]]]])

# Uporedite vaše rezultate sa našim. Razlika bi trebalo da je reda e-8.
print('Testiranje max_pool_forward_naive funkcije:')
print('razlika: ', rel_error(out, correct_out))

# Max-Pooling: Naivni prolaz unazad
mplementirajte prolaz unazad za operaciju konvolucije unutar funkcije `max_pool_backward_naive` koja se nalazi u `funkcije/layers.py`. Ponovo ne treba voditi računa o efikasnosti.

Uporedite dobijene rezultate sa numeričkim izvodom.

In [ ]:
np.random.seed(231)
x = np.random.randn(3, 2, 8, 8)
dout = np.random.randn(3, 2, 4, 4)
pool_param = {'pool_height': 2, 'pool_width': 2, 'stride': 2}

dx_num = eval_numerical_gradient_array(lambda x: max_pool_forward_naive(x, pool_param)[0], x, dout)

out, cache = max_pool_forward_naive(x, pool_param)
dx = max_pool_backward_naive(dout, cache)

# Greška bi trebalo da je e-12 ili manja 
print('Testiranje max_pool_backward_naive funkcije:')
print('dx error: ', rel_error(dx, dx_num))

# Brzi slojevi
Realizacija brzih pooling i konvolucionih slojeva nije lak zadatak. Pošto cilj ovog kursa nije optimizacija algoritama, u `funkcije/fast_layers.py` nalaze se gotove implementacije.

Implementacija brze konvolucije zavisi od Cython ekstenczije, da bi je podesili iz `funkcije` morate pokrenuti sljedeće:

```bash
python setup.py build_ext --inplace
```

**NAPOMENA:** Brza implementacija pooling sloja će biti efikasna samo ako se pooling regije ne preklapaju. Ako ovaj uslov nije zadovoljen, brzina ove implementacije neće se puno razlikovati od one koju ste vi realizovali.

Možete uporediti rezultate u brzini upotrebom sledećeg koda:

In [ ]:
# Relativne greške bi trebalo da su reda e-9 ili manje
from funkcije.fast_layers import conv_forward_fast, conv_backward_fast
from time import time
np.random.seed(231)
x = np.random.randn(100, 3, 31, 31)
w = np.random.randn(25, 3, 3, 3)
b = np.random.randn(25,)
dout = np.random.randn(100, 25, 16, 16)
conv_param = {'stride': 2, 'pad': 1}

t0 = time()
out_naive, cache_naive = conv_forward_naive(x, w, b, conv_param)
t1 = time()
out_fast, cache_fast = conv_forward_fast(x, w, b, conv_param)
t2 = time()

print('Testiranje conv_forward_fast:')
print('Naivni pristup: %fs' % (t1 - t0))
print('Brzi pristup: %fs' % (t2 - t1))
print('Ubrzanje: %fx' % ((t1 - t0) / (t2 - t1)))
print('Razlika: ', rel_error(out_naive, out_fast))

t0 = time()
dx_naive, dw_naive, db_naive = conv_backward_naive(dout, cache_naive)
t1 = time()
dx_fast, dw_fast, db_fast = conv_backward_fast(dout, cache_fast)
t2 = time()

print('\nTestiranje conv_backward_fast:')
print('Naivni pristup: %fs' % (t1 - t0))
print('Brzi pristup: %fs' % (t2 - t1))
print('Ubrzanje: %fx' % ((t1 - t0) / (t2 - t1)))
print('dx razlika: ', rel_error(dx_naive, dx_fast))
print('dw razlika: ', rel_error(dw_naive, dw_fast))
print('db razlika: ', rel_error(db_naive, db_fast))

In [ ]:
# Relativne greške bi trebalo da su blizu 0.0
from funkcije.fast_layers import max_pool_forward_fast, max_pool_backward_fast
np.random.seed(231)
x = np.random.randn(100, 3, 32, 32)
dout = np.random.randn(100, 3, 16, 16)
pool_param = {'pool_height': 2, 'pool_width': 2, 'stride': 2}

t0 = time()
out_naive, cache_naive = max_pool_forward_naive(x, pool_param)
t1 = time()
out_fast, cache_fast = max_pool_forward_fast(x, pool_param)
t2 = time()

print('Testiranje pool_forward_fast:')
print('Naivni pristup: %fs' % (t1 - t0))
print('Brzi pristup: %fs' % (t2 - t1))
print('Ubrzanje: %fx' % ((t1 - t0) / (t2 - t1)))
print('Razlika: ', rel_error(out_naive, out_fast))

t0 = time()
dx_naive = max_pool_backward_naive(dout, cache_naive)
t1 = time()
dx_fast = max_pool_backward_fast(dout, cache_fast)
t2 = time()

print('\nTestiranje pool_backward_fast:')
print('Naivni pristup: %fs' % (t1 - t0))
print('Brzi pristup: %fs' % (t2 - t1))
print('Ubrzanje: %fx' % ((t1 - t0) / (t2 - t1)))
print('dx razlika: ', rel_error(dx_naive, dx_fast))

# Konvolucioni "sendvič" slojevi
Prethodno smo uveli koncept "sendvič" slojeva koji kombinuju nekoliko operacija koje predstavljaju šablon. U `funkcije/layer_utils.py` se mogu naći sendvič slojevi sa par različitih šablona.

In [ ]:
from funkcije.layer_utils import conv_relu_pool_forward, conv_relu_pool_backward
np.random.seed(231)
x = np.random.randn(2, 3, 16, 16)
w = np.random.randn(3, 3, 3, 3)
b = np.random.randn(3,)
dout = np.random.randn(2, 3, 8, 8)
conv_param = {'stride': 1, 'pad': 1}
pool_param = {'pool_height': 2, 'pool_width': 2, 'stride': 2}

out, cache = conv_relu_pool_forward(x, w, b, conv_param, pool_param)
dx, dw, db = conv_relu_pool_backward(dout, cache)

dx_num = eval_numerical_gradient_array(lambda x: conv_relu_pool_forward(x, w, b, conv_param, pool_param)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: conv_relu_pool_forward(x, w, b, conv_param, pool_param)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: conv_relu_pool_forward(x, w, b, conv_param, pool_param)[0], b, dout)

# Relativne greške bi trebalo da su oko e-8 ili manje
print('Testiranje conv_relu_pool')
print('dx greška: ', rel_error(dx_num, dx))
print('dw greška: ', rel_error(dw_num, dw))
print('db greška: ', rel_error(db_num, db))

In [ ]:
from funkcije.layer_utils import conv_relu_forward, conv_relu_backward
np.random.seed(231)
x = np.random.randn(2, 3, 8, 8)
w = np.random.randn(3, 3, 3, 3)
b = np.random.randn(3,)
dout = np.random.randn(2, 3, 8, 8)
conv_param = {'stride': 1, 'pad': 1}

out, cache = conv_relu_forward(x, w, b, conv_param)
dx, dw, db = conv_relu_backward(dout, cache)

dx_num = eval_numerical_gradient_array(lambda x: conv_relu_forward(x, w, b, conv_param)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: conv_relu_forward(x, w, b, conv_param)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: conv_relu_forward(x, w, b, conv_param)[0], b, dout)

# Relativne greške bi trebalo da su oko e-8 ili manje
print('Testiranje conv_relu:')
print('dx greška: ', rel_error(dx_num, dx))
print('dw greška: ', rel_error(dw_num, dw))
print('db greška: ', rel_error(db_num, db))

# Troslojna konvoluciona mreža (ConvNet)
Nakon što ste implementirali sve potrebne slojeve, možemo ih spojiti u jednu cjelinu, troslojnu mrežu.

Otvorite `funkcije/classifiers/cnn.py` i završite implementaciju `ThreeLayerConvNet` klase. Ne zaboravite da možete koristi u implementaciji brze/sendvič slojeve koje ste već dobili.

Iskoristite sledeći kod da provjerite realizaciju:

## Provjera loss-a
Nakon što konstruišete mrežu, prva stvar koju treba provjeriti jeste loss funkcija. Kada koristimo softmax loss, očekujemo da za nasumične vrijednosti težina (bez regularizacije) vrijednost loss-a bude `log(C)` za `C` klasa. Kada dodamo regularizaciju ova vrijednost bi trebalo da se poveća

In [ ]:
model = ThreeLayerConvNet()

N = 50
X = np.random.randn(N, 3, 32, 32)
y = np.random.randint(10, size = N)

loss, grads = model.loss(X, y)
print('Inicijalni loss (bez regularizacije): ', loss)

model.reg = 0.5
loss, grads = model.loss(X, y)
print('Inicijalni loss (sa regularizacijom): ', loss)

## Provjera gradijenta
Ukoliko loss djeluje dobro, sledeće što treba uraditi jeste upotrebom numeričkog gradijenta provjeriti realizaciju prolaska unazad. Napomena: svi rezultati greške do e-2 bi trebalo da su dobri. 

In [ ]:
num_inputs = 2
input_dim = (3, 16, 16)
reg = 0.0
num_classes = 10
np.random.seed(231)
X = np.random.randn(num_inputs, *input_dim)
y = np.random.randint(num_classes, size = num_inputs)

model = ThreeLayerConvNet(num_filters = 3, filter_size = 3,
                          input_dim = input_dim, hidden_dim = 7,
                          dtype = np.float64)
loss, grads = model.loss(X, y)

for param_name in sorted(grads):
    f = lambda _: model.loss(X, y)[0]
    param_grad_num = eval_numerical_gradient(f, model.params[param_name], verbose = False, h = 1e-6)
    e = rel_error(param_grad_num, grads[param_name])
    print('%s maksimalna relativna greška: %e' % (param_name, rel_error(param_grad_num, grads[param_name])))

## Overfitovanje na malom broju ulaznih podataka
Kao još jedna od provjera jeste upotreba malog broja ulaznih podataka i velikog broja epoha. Ovim ćemo dobiti ogromnu preciznost na trening podacima, a značajno manju na validacionim podacima. Time dokazujemo da je mreža u stanju da uči.

In [ ]:
np.random.seed(231)

num_train = 100
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

model = ThreeLayerConvNet(weight_scale = 1e-2)

solver = Solver(model, small_data,
                num_epochs = 15, batch_size = 50,
                update_rule = 'adam',
                optim_config = {
                  'learning_rate': 1e-3,
                },
                verbose = True, print_every = 1)
solver.train()

Crtanje loss-a, trening preciznost i preciznosti na validacionim podacima će jasno pokazati da je došlo do overfita:

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(solver.loss_history, 'o')
plt.xlabel('iteracija')
plt.ylabel('loss')

plt.subplot(2, 1, 2)
plt.plot(solver.train_acc_history, '-o')
plt.plot(solver.val_acc_history, '-o')
plt.legend(['trening', 'validacija'], loc = 'upper left')
plt.xlabel('epoha')
plt.ylabel('preciznost')
plt.show()

## Treniranje mreže
Treniranjem konvolucione mreže u samo jednoj epohi bi trebalo da da preciznost veću od 40%:

In [ ]:
model = ThreeLayerConvNet(weight_scale=0.001, hidden_dim=500, reg=0.001)

solver = Solver(model, data,
                num_epochs = 1, batch_size = 50,
                update_rule = 'adam',
                optim_config = {
                  'learning_rate': 1e-3,
                },
                verbose = True, print_every = 20)
solver.train()

## Vizuelizacija filtera
Moguće je vizuelizovati prvi sloj filtera konvolucione mreže upotrebom sledećeg koda:

In [ ]:
from funkcije.vis_utils import visualize_grid

grid = visualize_grid(model.params['W1'].transpose(0, 2, 3, 1))
plt.imshow(grid.astype('uint8'))
plt.axis('off')
plt.gcf().set_size_inches(5, 5)
plt.show()

# Prostorna Batch Normalizacija
Kao što je već demonstrirano na nastavi, a i u originalnom radu [3], batch normalizacija (BN) se može koristiti za konvolucione mreže, ali se mora malo modifikovati. Ova modifikacija naziva se "prostornom batch normalizacijom". 

U slučaju FC slojeva, BN dobija ulazne podatke dimenzija `(N, D)`, a izlaz je takođe dimenzija `(N, D)` dok se normalizacija vrši po dimenziji `N`. U slučaju konvolucionog sloja, BN na svom ulazu dobija podatke dimenzija `(N, C, H, W)`. Izlaz je takođe dimenzija `(N, C, H, W)` gdje `N` dimenzija predstavlja veličinu mini batch-a, a `(H, W)` predstavlja prostorne dimenzije feature mape.

Ukoliko je feature mapa dobijena upotrebom konvolucije, tada očekujemo da će statistika svih feature kanala bili relativno konzistentna između različitih slika, ali i različitih lokacija na istoj slici. Zbog toga prostorna batch normalizacija računa srednju vrijednost i varijansu za svake `C` feature kanale tako što računa statistiku ne samo po dimenziji `N` već i po dimenzijama `H` i `W`.

[3] [Sergey Ioffe and Christian Szegedy, "Batch Normalization: Accelerating Deep Network Training by Reducing
Internal Covariate Shift", ICML 2015.](https://arxiv.org/abs/1502.03167)

## Prostorna BN: forward

U `funkcije/layers.py`, implementirajte prolaz unaprijed za prostornu BN unutar funkcije `spatial_batchnorm_forward`. Testirajte implementaciju pokretanjem sledećeg koda:

In [ ]:
np.random.seed(231)

# Provjerite prolaz unaprijed tokom treniranja tako što ćete provjeriti srednju
# vrijednost i varijansu parametara kako prije tako i poslije prostorne BN

N, C, H, W = 2, 3, 4, 5
x = 4 * np.random.randn(N, C, H, W) + 10

print('Prije prostorne BN:')
print('  Dimenzije: ', x.shape)
print('  Srednje vr.: ', x.mean(axis = (0, 2, 3)))
print('  Standardna dev.: ', x.std(axis = (0, 2, 3)))

# Srednje vrijednosti bi trebalo da su blizu nula, a standardne devijacije oko jedan
gamma, beta = np.ones(C), np.zeros(C)
bn_param = {'mode': 'train'}
out, _ = spatial_batchnorm_forward(x, gamma, beta, bn_param)
print('Nakon prostorne BN:')
print('  Dimenzije: ', out.shape)
print('  Srednje vr.: ', out.mean(axis = (0, 2, 3)))
print('  Standardna dev.: ', out.std(axis = (0, 2, 3)))

# Srednje vrijednosti bi trebalo da su blizu beta, a standardne devijacije oko gamma
gamma, beta = np.asarray([3, 4, 5]), np.asarray([6, 7, 8])
out, _ = spatial_batchnorm_forward(x, gamma, beta, bn_param)
print('Nakon prostorne BN (netrivijalne beta i gamma):')
print('  Dimenzije: ', out.shape)
print('  Srednje vr.: ', out.mean(axis = (0, 2, 3)))
print('  Standardna dev.: ', out.std(axis = (0, 2, 3)))

In [ ]:
np.random.seed(231)
# Sada ćemo provjeriti rad u režimu testiranja tako što ćemo proći
# kroz režim treniranja više puta kako bismo "zagrijali" srednje vrijednosti
# a onda provjerili rezultate

N, C, H, W = 10, 4, 11, 12

bn_param = {'mode': 'train'}
gamma = np.ones(C)
beta = np.zeros(C)
for t in range(50):
    x = 2.3 * np.random.randn(N, C, H, W) + 13
    spatial_batchnorm_forward(x, gamma, beta, bn_param)
bn_param['mode'] = 'test'
x = 2.3 * np.random.randn(N, C, H, W) + 13
a_norm, _ = spatial_batchnorm_forward(x, gamma, beta, bn_param)

# Srednje vrijednosti bi trebalo da su blizu nula, a standardne devijacije oko jedan, ali sa određenim šumom
print('Nakon prostorne BN (režim testiranja):')
print('  Srednje vr.: ', a_norm.mean(axis = (0, 2, 3)))
print('  Standardna dev.: ', a_norm.std(axis = (0, 2, 3)))

## Prostorna BN: backward
U `funkcije/layers.py`, implementirajte prolaz unazad za prostornu BN unutar funkcije `spatial_batchnorm_backward`. Testirajte svoju implementaciju poređenjem sa numeričkim gradijentom:

In [ ]:
np.random.seed(231)
N, C, H, W = 2, 3, 4, 5
x = 5 * np.random.randn(N, C, H, W) + 12
gamma = np.random.randn(C)
beta = np.random.randn(C)
dout = np.random.randn(N, C, H, W)

bn_param = {'mode': 'train'}
fx = lambda x: spatial_batchnorm_forward(x, gamma, beta, bn_param)[0]
fg = lambda a: spatial_batchnorm_forward(x, gamma, beta, bn_param)[0]
fb = lambda b: spatial_batchnorm_forward(x, gamma, beta, bn_param)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
da_num = eval_numerical_gradient_array(fg, gamma, dout)
db_num = eval_numerical_gradient_array(fb, beta, dout)

#Greška bi trebalo da je reda 1e-12~1e-06
_, cache = spatial_batchnorm_forward(x, gamma, beta, bn_param)
dx, dgamma, dbeta = spatial_batchnorm_backward(dout, cache)
print('dx greška: ', rel_error(dx_num, dx))
print('dgamma greška: ', rel_error(da_num, dgamma))
print('dbeta greška: ', rel_error(db_num, dbeta))

# Grupna normalizacija

Ranije je pominjana sloj normalizacija (Layer Normalization) kao alternativna tehnika koja anulira neke od mana batch normalizacije. Kao što su autori [4] zaključili, LN ne daje dobre rezultate kao BN u konvolucionim neuralnim mrežama:

> Kod FC slojeva, svi skriveni neuroni teže da imaju isti doprinos pri računanju konačne predikcije, iz tog razloga centriranje i skaliranje ulaza u neuron radi očekivano dobro. Međutim, ista pretpostavka ne važi kod konvolucionih neuralnih mreža. Kako dubina konvolucione neuralne mreže raste tako raste i broj neurona čije receptivno polje pada blizu okvira slike i rijetko su "upaljeni" (množe vrijednost različitu od nule), i zbog toga imaju veoma različitu statistiku od ostatka neurona u istom sloju.

Autori [5] predlažu neku vrstu prelaznog rješenja. Za razliku od LN gdje se normalizuju svi featuri po jednom ulazu, ideja ovih autora je da se jedna feature mapa podijeli u G podgrupa na kojim će se vršiti normalizacija.  

![Comparison of normalization techniques discussed so far](normalization.png)
<center>**Vizuelno poređenje diskutovanih metoda (slika iz [5])**</center>

Vaš zadatak je da sada implementirate grupnu normalizaciju. Imajte na umu da je BN metoda stara četiri godine i već ustaljena, dok je GN metoda razvijena prije nešto više od godinu dana!

[4] [Ba, Jimmy Lei, Jamie Ryan Kiros, and Geoffrey E. Hinton. "Layer Normalization." stat 1050 (2016): 21.](https://arxiv.org/pdf/1607.06450.pdf)


[5] [Wu, Yuxin, and Kaiming He. "Group Normalization." arXiv preprint arXiv:1803.08494 (2018).](https://arxiv.org/abs/1803.08494)


## Grupna normalizacija: forward

U `funkcije/layers.py`, implementirajte prolaz unaprijed za grupnu normalizaciju unutar funkcije `spatial_groupnorm_forward`. Testirajte svoju implementaciju upotrebom sledećeg koda:

In [ ]:
np.random.seed(231)

# Provjerite prolaz unaprijed tokom treniranja tako što ćete provjeriti srednju
# vrijednost i varijansu parametara kako prije tako i poslije prostorne GN

N, C, H, W = 2, 6, 4, 5
G = 2
x = 4 * np.random.randn(N, C, H, W) + 10
x_g = x.reshape((N*G,-1))

print('Prije prostorne GN:')
print('  Dimenzije: ', x.shape)
print('  Srednje vr.: ', x_g.mean(axis = 1))
print('  Standardna dev.: ', x_g.std(axis = 1))

# Srednje vrijednosti bi trebalo da su blizu nula, a standardne devijacije oko jedan
gamma, beta = np.ones((1,C,1,1)), np.zeros((1,C,1,1))
bn_param = {'mode': 'train'}

out, _ = spatial_groupnorm_forward(x, gamma, beta, G, bn_param)
out_g = out.reshape((N*G,-1))
print('Nakon prostorne GN:')
print('  Dimenzije: ', out.shape)
print('  Srednje vr.: ', out_g.mean(axis = 1))
print('  Standardna dev.: ', out_g.std(axis = 1))

## Grupna normalizacija: backward

U `funkcije/layers.py`, implementirajte prolaz unazad za grupnu normalizaciju unutar funkcije `spatial_groupnorm_backward`. Testirajte svoju implementaciju poređenjem sa numeričkim gradijentom:

In [ ]:
np.random.seed(231)
N, C, H, W = 2, 6, 4, 5
G = 2
x = 5 * np.random.randn(N, C, H, W) + 12
gamma = np.random.randn(1,C,1,1)
beta = np.random.randn(1,C,1,1)
dout = np.random.randn(N, C, H, W)

gn_param = {}
fx = lambda x: spatial_groupnorm_forward(x, gamma, beta, G, gn_param)[0]
fg = lambda a: spatial_groupnorm_forward(x, gamma, beta, G, gn_param)[0]
fb = lambda b: spatial_groupnorm_forward(x, gamma, beta, G, gn_param)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
da_num = eval_numerical_gradient_array(fg, gamma, dout)
db_num = eval_numerical_gradient_array(fb, beta, dout)

_, cache = spatial_groupnorm_forward(x, gamma, beta, G, gn_param)
dx, dgamma, dbeta = spatial_groupnorm_backward(dout, cache)

#Greška bi trebalo da je reda 1e-12~1e-07
print('dx greška: ', rel_error(dx_num, dx))
print('dgamma greška: ', rel_error(da_num, dgamma))
print('dbeta greška: ', rel_error(db_num, dbeta))